In [1]:
import requests
import pymysql
from bs4 import BeautifulSoup as bs
import json

In [2]:
def dataDeal(content):
    db = pymysql.connect(host = 'localhost',user = 'root',passwd ='123456',db = 'cfets',charset='utf8' )
    cursor = db.cursor()
    try:
        cursor.execute(content)
        print(cursor.fetchall())
        db.commit()
        db.close()
    except:
        db.close()

In [3]:
class Shibor3M(object):
    """本类用于从中国货币网爬取Shibor 3M数据"""
    def __init__(self, url, headers, cookies):
        self.url = url
        self.headers = headers
        self.cookies = cookies
        
    def getJson(self):
        sourcejson = requests.post(url = self.url,headers = self.headers,cookies = self.cookies)
        if sourcejson.status_code != 200:
            return
        sourcejson = sourcejson.content
        # sourcejson = sourcejson.decode("utf-8")
        jsonSource = json.loads(sourcejson)
        for k,v in jsonSource.items():
            # print(k, type(v), v)
            pass
        jsonDict = {}
        jsonDict["curveType"] = jsonSource["data"]["curveType"]
        jsonDict["curveTypeEn"] = jsonSource["data"]["curveTypeEn"]
        jsonDict["cfgItemType"] = jsonSource["data"]["cfgItemType"] #curveId
        jsonDict["curveId"] = jsonSource["data"]["curveId"] #curveId
        try:
            jsonDict["options"] = jsonSource["data"]["options"] [jsonDict["curveId"]] or ""#curveId
        except:
            jsonDict["options"] = ""
        jsonDict["quoteTime"] = jsonSource["data"]["quoteTime"]
        jsonDict["showDateCN"] = jsonSource["data"]["showDateCN"]
        jsonDict["showDateEN"] = jsonSource["data"]["showDateEN"]
        jsonDict["provider"] = jsonSource["head"]["provider"]
        jsonDict["ts"] = jsonSource["head"]["ts"]
        jsonDict["tstext"] = jsonSource["head"]["tstext"]
        try:
            jsonDict["records"] = jsonSource["records"]
        except:
            jsonDict["records"] = ""
        jsonDict["issueCode"] = {}
        if jsonDict["records"] != "":
            for index,line in enumerate(jsonDict["records"]):
                jsonDict["issueCode"][jsonDict["curveTypeEn"] + "_" + line["tl"]] = [line["tl"],line["optimalAvg"],line["optimalAsk"],line["optimalBid"],line["curveid"]]        
                for k,v in jsonDict["issueCode"].items():
                    # print(k,jsonDict["showDateCN"],jsonDict["quoteTime"],jsonDict["curveTypeEn"],jsonDict["cfgItemType"],jsonDict["curveId"],jsonDict["options"],jsonDict["provider"],jsonDict["tstext"],v,)
                    str = "insert into IRS_CFETS(IssueCode,showDateCN,quoteTime,curveTypeEn,cfgItemType,curveId,options,provider,tstext,tl,optimalAvg,optimalAsk,optimalBid) value('{IssueCode}','{showDateCN}','{quoteTime}','{curveTypeEn}','{cfgItemType}','{curveId}','{options}','{provider}','{tstext}','{tl}','{optimalAvg}','{optimalAsk}','{optimalBid}');"\
                        .format(IssueCode = k,showDateCN = jsonDict["showDateCN"],quoteTime = jsonDict["quoteTime"],curveTypeEn = jsonDict["curveTypeEn"],cfgItemType = jsonDict["cfgItemType"],curveId = jsonDict["curveId"],options = jsonDict["options"],provider = jsonDict["provider"],tstext = jsonDict["tstext"],tl = v[0] ,optimalAvg = v[1],optimalAsk = v[2],optimalBid = v[3])
                    print(str)
                    dataDeal(str)

In [13]:
url = r"""http://www.chinamoney.com.cn/ags/ms/cm-u-bk-shibor/IfccHis?cfgItemType=71&lang=EN"""
headers = {
    "Accept": "application/json,text/javascript,*/*;q=0.01",
    "Accept-Encoding":"gzip,deflate",
    "Accept-Language":"zh-CN,zh;q=0.9",
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36"
}
data = {"startDate": "2019-02-14", 
        "endDate": "2019-03-14",
        "interestRateType": "16",
        "quoteTime": "16:30",
        "bidAskType": "avg"}

In [14]:
res = requests.post(url, headers=headers, data=data)

In [15]:
resJson = json.loads(res.content)

In [16]:
resJson

{'head': {'version': '2.0',
  'provider': 'CWAP',
  'req_code': '',
  'rep_code': '200',
  'rep_message': '',
  'ts': 1552614821197,
  'producer': '',
  'tstext': '2019-03-15 09:53:41'},
 'data': {'curveids': ['15', '17', '16'],
  'message': '',
  'endDateEN': '14 Mar 2019',
  'bightNamelist': ['Shibor3M Fixing Curve',
   'Shibor3M Quotes Curve',
   'Shibor3M Closing Curve'],
  'quoteTime': '16:30',
  'startDateCN': '2019-02-14',
  'cfgItemType': '71',
  'quoteTimeList': ['16:30'],
  'endDateCN': '2019-03-14',
  'curveid': '16',
  'bigthRowName': ['6M', '9M', '1Y', '2Y', '3Y', '4Y', '5Y', '7Y', '10Y'],
  'startDateEN': '14 Feb 2019',
  'shortNameList': ['Fixing', 'Quotes', 'Closing']},
 'records': [{'curveType': 'Closing Curve',
   'bigthType': 'CISCS3M',
   'dateShowEn': '14 Mar 2019',
   'bigthName': 'Shibor3M Closing Curve',
   'quoteTime': '16:30',
   'configPrice': ['2.8988',
    '2.9775',
    '3.0413',
    '3.1800',
    '3.3000',
    '3.4063',
    '3.5050',
    '3.6750',
    '3.8